In [1]:
import numpy as np
import pandas as pd

In [2]:
trainDf = pd.read_csv('./train.csv')

In [3]:
labels = trainDf['label'].values

In [4]:
X = trainDf.iloc[:,1:].values

In [5]:
X.shape

(42000, 784)

In [6]:
X = X/255

In [7]:
X = X.reshape((X.shape[0], 28, 28, 1))

In [8]:
from keras import Sequential
from keras.layers import Dense, Softmax, Reshape, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.losses import MSE
from keras.utils.np_utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [9]:
import keras.backend as K

In [10]:
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

In [11]:
import math
def get_learning_rate(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
    

In [12]:
model = Sequential()
#model.add(Reshape((28,28,1), input_shape=(784,)))
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu', padding='Same'))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
learning_rate_scheduler = LearningRateScheduler(get_learning_rate)
reduce_lr_callback = ReduceLROnPlateau(factor=0.5, min_lr=0.00001, patience=3, verbose=1)

In [14]:
trainingImageGen = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, shear_range=0.2, zoom_range=0.2, height_shift_range=0.2, vertical_flip=False, horizontal_flip=False)
validImageGen = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, shear_range=0.2, zoom_range=0.2, height_shift_range=0.2, vertical_flip=False, horizontal_flip=False)

In [15]:
trainingImageGen.fit(X)
validImageGen.fit(X)

In [16]:
trainingGenerator = trainingImageGen.flow(X, to_categorical(labels),batch_size=32)
validImageGenerator = validImageGen.flow(X, to_categorical(labels),batch_size=32)

In [17]:
help(model.fit_generator)

Help on method fit_generator in module keras.engine.training:

fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0) method of keras.engine.sequential.Sequential instance
    Trains the model on data generated batch-by-batch by a Python generator
    (or an instance of `Sequence`).
    
    The generator is run in parallel to the model, for efficiency.
    For instance, this allows you to do real-time data augmentation
    on images on CPU in parallel to training your model on GPU.
    
    The use of `keras.utils.Sequence` guarantees the ordering
    and guarantees the single use of every input per epoch when
    using `use_multiprocessing=True`.
    
    # Arguments
        generator: A generator or an instance of `Sequence`
            (`keras.utils.Sequence`) object in order to avoid
            duplica

In [21]:
#model.fit(xTrain, to_categorical(yTrain), epochs=10, batch_size=256)
history = model.fit_generator(trainingGenerator, steps_per_epoch=X.shape[0]//32, epochs=30, callbacks=[reduce_lr_callback], validation_data=validImageGenerator, validation_steps=X.shape[0]//32)

Epoch 1/30
1312/1312 [==============================] - 117s 89ms/step - loss: 0.3855 - acc: 0.8779 - val_loss: 0.1625 - val_acc: 0.9517
Epoch 2/30
1312/1312 [==============================] - 116s 89ms/step - loss: 0.1305 - acc: 0.9634 - val_loss: 0.1052 - val_acc: 0.9699
Epoch 3/30
1312/1312 [==============================] - 117s 89ms/step - loss: 0.1050 - acc: 0.9699 - val_loss: 0.0842 - val_acc: 0.9771
Epoch 4/30
1312/1312 [==============================] - 113s 86ms/step - loss: 0.0965 - acc: 0.9737 - val_loss: 0.0886 - val_acc: 0.9741
Epoch 5/30
1312/1312 [==============================] - 113s 86ms/step - loss: 0.0861 - acc: 0.9766 - val_loss: 0.0813 - val_acc: 0.9757
Epoch 6/30
1312/1312 [==============================] - 113s 86ms/step - loss: 0.0819 - acc: 0.9781 - val_loss: 0.0981 - val_acc: 0.9705
Epoch 7/30
1312/1312 [==============================] - 113s 86ms/step - loss: 0.0773 - acc: 0.9791 - val_loss: 0.1235 - val_acc: 0.9657
Epoch 8/30
1312/1312 [===================

In [26]:
history = model.fit_generator(trainingGenerator, steps_per_epoch=X.shape[0]//32, epochs=6, callbacks=[reduce_lr_callback], validation_data=validImageGenerator, validation_steps=X.shape[0]//32)

Epoch 1/6
1312/1312 [==============================] - 132s 100ms/step - loss: 0.0186 - acc: 0.9946 - val_loss: 0.0174 - val_acc: 0.9948
Epoch 2/6
1312/1312 [==============================] - 124s 94ms/step - loss: 0.0180 - acc: 0.9945 - val_loss: 0.0164 - val_acc: 0.9951
Epoch 3/6
1312/1312 [==============================] - 116s 88ms/step - loss: 0.0179 - acc: 0.9950 - val_loss: 0.0154 - val_acc: 0.9949
Epoch 4/6
1312/1312 [==============================] - 114s 87ms/step - loss: 0.0170 - acc: 0.9953 - val_loss: 0.0165 - val_acc: 0.9951
Epoch 5/6
1312/1312 [==============================] - 117s 89ms/step - loss: 0.0193 - acc: 0.9950 - val_loss: 0.0148 - val_acc: 0.9955
Epoch 6/6
1312/1312 [==============================] - 116s 89ms/step - loss: 0.0183 - acc: 0.9945 - val_loss: 0.0152 - val_acc: 0.9954


In [31]:
history = model.fit_generator(trainingGenerator, steps_per_epoch=X.shape[0]//32, epochs=10, callbacks=[reduce_lr_callback], validation_data=validImageGenerator, validation_steps=X.shape[0]//32)

Epoch 1/10
1312/1312 [==============================] - 118s 90ms/step - loss: 0.0169 - acc: 0.9951 - val_loss: 0.0152 - val_acc: 0.9953
Epoch 2/10
1312/1312 [==============================] - 116s 88ms/step - loss: 0.0159 - acc: 0.9950 - val_loss: 0.0160 - val_acc: 0.9951
Epoch 3/10
1312/1312 [==============================] - 115s 88ms/step - loss: 0.0171 - acc: 0.9952 - val_loss: 0.0143 - val_acc: 0.9956
Epoch 4/10
1312/1312 [==============================] - 117s 89ms/step - loss: 0.0176 - acc: 0.9946 - val_loss: 0.0146 - val_acc: 0.9955
Epoch 5/10
1312/1312 [==============================] - 118s 90ms/step - loss: 0.0156 - acc: 0.9954 - val_loss: 0.0148 - val_acc: 0.9955
Epoch 6/10
1312/1312 [==============================] - 118s 90ms/step - loss: 0.0169 - acc: 0.9951 - val_loss: 0.0140 - val_acc: 0.9953
Epoch 7/10
1312/1312 [==============================] - 116s 89ms/step - loss: 0.0156 - acc: 0.9956 - val_loss: 0.0137 - val_acc: 0.9957
Epoch 8/10
1312/1312 [===================

In [32]:
history = model.fit_generator(trainingGenerator, steps_per_epoch=X.shape[0]//32, epochs=10, callbacks=[reduce_lr_callback], validation_data=validImageGenerator, validation_steps=X.shape[0]//32)

Epoch 1/10
1312/1312 [==============================] - 116s 88ms/step - loss: 0.0136 - acc: 0.9960 - val_loss: 0.0114 - val_acc: 0.9963
Epoch 2/10
1312/1312 [==============================] - 116s 88ms/step - loss: 0.0146 - acc: 0.9960 - val_loss: 0.0126 - val_acc: 0.9963
Epoch 3/10
1312/1312 [==============================] - 114s 87ms/step - loss: 0.0124 - acc: 0.9960 - val_loss: 0.0125 - val_acc: 0.9962
Epoch 4/10
1312/1312 [==============================] - 115s 88ms/step - loss: 0.0130 - acc: 0.9962 - val_loss: 0.0124 - val_acc: 0.9963

Epoch 00004: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 5/10
1312/1312 [==============================] - 114s 87ms/step - loss: 0.0124 - acc: 0.9966 - val_loss: 0.0117 - val_acc: 0.9966
Epoch 6/10
1312/1312 [==============================] - 113s 86ms/step - loss: 0.0137 - acc: 0.9958 - val_loss: 0.0105 - val_acc: 0.9966
Epoch 7/10
1312/1312 [==============================] - 114s 87ms/step - loss: 0.0116 - acc: 0.99

In [37]:
history = model.fit_generator(trainingGenerator, steps_per_epoch=X.shape[0]//32, epochs=10, callbacks=[reduce_lr_callback], validation_data=validImageGenerator, validation_steps=X.shape[0]//32)

Epoch 1/10
1312/1312 [==============================] - 118s 90ms/step - loss: 0.0129 - acc: 0.9963 - val_loss: 0.0112 - val_acc: 0.9965
Epoch 2/10
1312/1312 [==============================] - 119s 91ms/step - loss: 0.0114 - acc: 0.9968 - val_loss: 0.0107 - val_acc: 0.9969
Epoch 3/10
1312/1312 [==============================] - 122s 93ms/step - loss: 0.0126 - acc: 0.9963 - val_loss: 0.0108 - val_acc: 0.9966
Epoch 4/10
1312/1312 [==============================] - 123s 94ms/step - loss: 0.0128 - acc: 0.9965 - val_loss: 0.0103 - val_acc: 0.9969
Epoch 5/10
1312/1312 [==============================] - 124s 95ms/step - loss: 0.0116 - acc: 0.9968 - val_loss: 0.0120 - val_acc: 0.9962
Epoch 6/10
1312/1312 [==============================] - 121s 92ms/step - loss: 0.0108 - acc: 0.9968 - val_loss: 0.0099 - val_acc: 0.9967
Epoch 7/10
1312/1312 [==============================] - 121s 92ms/step - loss: 0.0125 - acc: 0.9964 - val_loss: 0.0109 - val_acc: 0.9968
Epoch 8/10
1312/1312 [===================

In [38]:
testDf = pd.read_csv('test.csv')

In [39]:
X_test = testDf.values
#normalizing
X_test = X_test/255
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

In [40]:
Y = model.predict_classes(X_test)

In [41]:
with open('test_resultCNNImageAugmentationNormalized.csv', 'w') as fil:
    print('ImageId,Label', file=fil)
    for i in range(Y.shape[0]):
        print(i+1, Y[i], sep=',', file=fil)